Para el correcto funcionamiento de este fichero es necesario crear dentro de Colab las carpetas ods, csv, txt y pdf. Dentro de las carpetas pdf y ods deberán estar todos los ficheros a transformar.
En este caso, he modificado los fichero txt que extraía para hacer más sencilla la transformación, esto lo he hecho porque eran pocas páginas por documento pdf y sólo 4 documentos.

In [1]:
import pandas as pd
!pip install tika
from tika import parser
from google.colab import files
!pip install odfpy

  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32625 sha256=0ce892bcab7a5d8a8905e6ff0c4315016f1b56cbd2ae5419faa7ee16203b99d9
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160672 sha256=0d4ea9c76d76d0f92ca404c497eb6b36c381eee2503f4fbb2174fa8820142c5b
  Stored in directory: /root/.cache/pip/wheels/c8/2e/95/90d94fe33903786937f3b8c33dd88807f792359c6424b40469
Successfully built odfpy


In [2]:
def init():

  # opening pdf file
  parsed_pdf = parser.from_file(ruta + "pdf/" + document + ".pdf")

  # saving content of pdf
  # you can also bring text only, by parsed_pdf['text']
  # parsed_pdf['content'] returns string
  data = parsed_pdf['content']

  # writing to file
  # with open(ruta + "txt/" + document + ".txt", 'w') as f:
      # f.write(data)

  # reading from file
  file1 = open(ruta + "txt/" + document + ".txt", 'r')
  Lines = file1.readlines()
  extractPages(pagina, pages, Lines)
  # extractPage([], pages, 3, Lines)
  # extractPage([], pages, 5, Lines)

In [3]:
# Crea listas de cada página
def extractPages(myList, pages, Lines):
  listaSinEspacios = []
  # Formateamos y quitamos todas las líneas vacías
  for line in Lines:
    if line != '\n' and line != ' \n':
      listaSinEspacios.append(line)
  Lines = listaSinEspacios
  for line in Lines:
    if line.find("de Alicante") != -1:
      pages +=1
      if pages !=0:
        # writeTxt(myList, pages)
        extractDataRenunciados(myList, pages)
        myList = []
    myList.append(line)
  # writeTxt(myList, pages+1)
  extractDataRenunciados(myList, pages+1)

In [4]:
# Crea un documento txt de la primera página

def writeTxt(data_List, page):
  data_Str = ' '.join(data_List)
  with open(ruta + "txt/" + document + "_"+ str(page) + ".txt", 'w') as f:
    f.write(data_Str)

In [5]:
# Extrae los datos de una página de los contratos renunciados guardada en una lista de Strings

"""
 columnas = ["Num. Exp",
              "Objeto Contrato",
              "Organismo",
              "Tipo de Contrato",
              "Tipo Tramitación",
              "Importe",
              "Estado",
              "Causa"]
"""

def extractDataRenunciados(pagina, pages):
  # Inicializamos las variables para poder usarlas en cada llamada e introducirlas en el DataSet tras el bucle For
  numExpediente = objContrato = organismo  = tipoContrato = tipoTramitacion = " "
  importe = estado = causa = linea2 =  " "

  # print("Aqui se extraerían los datos de la página " + str(pages))
  for count, value in enumerate(pagina):
    if "Nº Expediente:" in value:
      numExpediente = value[(value.find("Nº Expediente:")) + 15:]
    if "Objeto del contrato:" in value:
      idObjContrato = count - 1
      while ("Entre el" not in pagina[idObjContrato] and idObjContrato >= 0):
        idObjContrato -= 1
      for x in range(idObjContrato+1, count):
        linea2 = pagina[x]
        objContrato += linea2
      objContrato = objContrato.replace('\n', " ")
      # print(objContrato)
    if "Órgano gestor:" in value:
      organismo = value[(value.find("Órgano gestor:") + 15):(value.find("Nº Expediente:"))]
    if "Tramitación:" in value:
      tipoContrato =  value[: (value.find("Tramitación:"))]
      tipoTramitacion = value[(value.find("Tramitación:") + 13) :]
    if "Importe adjudicación sin I.V.A.:" in value:
      importe = value[(value.find("con I.V.A.:")) + 12 :]
      print
    if "Estado:" in value:
      estado = value[(value.find("Estado: ")) + 8 :]
    if "Causa:" in value:
      causa = value[(value.find("Causa: ")) + 7 :]

  # Creamos la lista fila con todos los datos de una fila del DataSet e introducimos la fila en el fichero, esto se repite en los x llamadas que se hacen a la función,
  # x equivalente al número de páginas del documento
  fila = [numExpediente, objContrato, organismo, tipoContrato, tipoTramitacion, importe, estado, causa]
  renunciados.loc[len(renunciados)] = fila
  renunciados.to_csv(ruta + "csv/renunciados_" + str(anyo) + ".csv")

In [6]:
# Convierte los archivos ODS a CSV para poder juntar todos en el mismo formato
def ods_to_csv():
  for anyo in range (2019, 2023):
    # print(str(anyo))
    columnas = ["Num. Exp",
              "Objeto Contrato",
              "Organismo",
              "Tipo de Contrato",
              "Tipo Tramitación",
              "Importe",
              "Estado",
              "Causa"]
    columnasODS = [ 'Num. Exp',
                   'Objeto Contrato',
                    "Tipo de Contrato",
                    "Tipo Tramitación",
                    "Borrar",
                    "Borrar",
                    "Importe",
                    "Borrar",
                    "Estado"]

    ruta = "/content/"
    document = "ods/renunciados_" + str(anyo) + ".ods"

    if anyo == 2019:
      renunciados = pd.read_excel(ruta + document, engine="odf", header=1)
      renunciados.columns = columnasODS
      renunciados = renunciados.drop(columns="Borrar")
      renunciados.insert(0, "Organismo", "NaN")
      renunciados.insert(0, "Causa", "NaN")
      renunciados = renunciados.reindex(columns=columnas)

    if anyo == 2020:
      renunciados = pd.read_excel(ruta + document, engine="odf", header=2)
      columnasODS = [ 'Num. Exp',
                   'Objeto Contrato',
                    "Tipo de Contrato",
                    "Tipo Tramitación",
                    "Borrar",
                    "Borrar",
                    "Importe",
                    "Estado"]
      renunciados.columns = columnasODS
      renunciados = renunciados.drop(columns="Borrar")
      renunciados.insert(0, "Organismo", "NaN")
      renunciados.insert(0, "Causa", "NaN")
      renunciados = renunciados.reindex(columns=columnas)

    if anyo == 2021:
      renunciados = pd.read_excel(ruta + document, engine="odf", header=3)
      renunciados = renunciados.drop(columns="Unnamed: 2")
      renunciados = renunciados.drop(columns="Unnamed: 4")
      renunciados = renunciados.drop(columns="Unnamed: 8")
      columnasODS = [ 'Num. Exp',
                      'Objeto Contrato',
                      "Tipo de Contrato",
                      "Tipo Tramitación",
                      "Borrar",
                      "Borrar",
                      "Importe",
                      "Borrar",
                      "Borrar",
                      "Borrar",
                      "Borrar"]
      renunciados.columns = columnasODS
      renunciados = renunciados.drop(columns="Borrar")
      renunciados.insert(0, "Organismo", "NaN")
      renunciados.insert(0, "Causa", "NaN")
      renunciados.insert(0, "Estado", "Desierto")
      renunciados = renunciados.reindex(columns=columnas)

    if anyo == 2022:
      renunciados = pd.read_excel(ruta + document, engine="odf", header=3)
      renunciados = renunciados.drop(columns="Unnamed: 2")
      renunciados = renunciados.drop(columns="Unnamed: 9")
      columnasODS = [ 'Num. Exp',
                      'Objeto Contrato',
                      "Borrar",
                      "Tipo de Contrato",
                      "Tipo Tramitación",
                      "Borrar",
                      "Importe",
                      "Borrar",
                      "Borrar",
                      "Borrar",
                      "Borrar"]
      renunciados.columns = columnasODS
      renunciados = renunciados.drop(columns="Borrar")
      renunciados.insert(0, "Organismo", "NaN")
      renunciados.insert(0, "Causa", "NaN")
      renunciados.insert(0, "Estado", "Desierto")
      renunciados = renunciados.reindex(columns=columnas)

    renunciados.to_csv(ruta + "csv/renunciados_" + str(anyo) + ".csv")

In [7]:
# Une los contratos de todos los años, añadiendo una columna "Año" en un archivo CSV
def unir_renunciados():
  union = pd.DataFrame()
  for anyo in range (2015, 2023):
    ruta = "/content/"
    document = "csv/renunciados_" + str(anyo) + ".csv"

    renunciados = pd.read_csv(ruta+document)
    renunciados = renunciados.drop(columns="Unnamed: 0")

    renunciados.insert(len(columnas), "Año", anyo)
    union = pd.concat([union, renunciados])


    # display(union)
  union.to_csv(ruta + "renunciados" + ".csv")

In [8]:
# Ejecutar todo
for anyo in range (2015, 2019):
  # anyo = 2015
  pagina = []
  pages = -1
  columnas = ["Num. Exp",
              "Objeto Contrato",
              "Organismo",
              "Tipo de Contrato",
              "Tipo Tramitación",
              "Importe",
              "Estado",
              "Causa"]
  renunciados = pd.DataFrame(columns = columnas)
  ruta = "/content/"
  document = "contratos-renunciados-" + str(anyo);
  init()

ods_to_csv()

2023-06-29 07:56:00,401 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /tmp/tika-server.jar.
2023-06-29 07:56:00,910 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
INFO:tika.tika:Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /tmp/tika-server.jar.md5.
2023-06-29 07:56:01,235 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2023-06-29 07:56:06,252 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [9]:
unir_renunciados()

In [10]:
for x in range (2015, 2023):
  files.download("/content/csv/renunciados_" + str(x) + ".csv")
  # files.download("/content/txt/contratos-renunciados-" + str(x) + ".txt")
files.download("/content/renunciados" + ".csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>